In [9]:
import torch
import matplotlib.pyplot as plt
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import _pickle as pickle
import numpy as np
import os

In [20]:
def get_CIFAR10_data(cifar10_dir, num_training=49000, num_validation=1000, num_test=1000):
    '''
    Load the CIFAR-10 dataset from disk and perform preprocessing to prepare
    it for the neural net classifier.
    '''
    # Load the raw CIFAR-10 data
    X_train, y_train, X_test, y_test = load(cifar10_dir)

    # Subsample the data
    mask = range(num_training, num_training + num_validation)
    X_val = X_train[mask]
    y_val = y_train[mask]
    mask = range(num_training)
    X_train = X_train[mask]
    y_train = y_train[mask]
    mask = range(num_test)
    X_test = X_test[mask]
    y_test = y_test[mask]

    X_train = X_train.astype(np.float64)
    X_val = X_val.astype(np.float64)
    X_test = X_test.astype(np.float64)

    # Transpose so that channels come first
    X_train = X_train.transpose(0, 3, 1, 2)
    X_val = X_val.transpose(0, 3, 1, 2)
    X_test = X_test.transpose(0, 3, 1, 2)
    mean_image = np.mean(X_train, axis=0)
    std = np.std(X_train)

    X_train -= mean_image
    X_val -= mean_image
    X_test -= mean_image

    X_train /= std
    X_val /= std
    X_test /= std

    return {
        'X_train': X_train, 'y_train': y_train,
        'X_val': X_val, 'y_val': y_val,
        'X_test': X_test, 'y_test': y_test,
        'mean': mean_image, 'std': std
    }


def load_CIFAR_batch(filename):
    ''' load single batch of cifar '''
    with open(filename, 'rb') as f:
        datadict = pickle.load(f, encoding ='bytes')
        X = datadict[b'data']
        Y = datadict[b'labels']
        X = X.reshape(10000, 3, 32, 32).transpose(0, 2, 3, 1)
        Y = np.array(Y)
        return X, Y


def load(ROOT):
    ''' load all of cifar '''
    xs = []
    ys = []
    for b in range(1, 6):
        f = os.path.join(ROOT, 'data_batch_%d' % (b, ))
        X, Y = load_CIFAR_batch(f)
        xs.append(X)
        ys.append(Y)
    Xtr = np.concatenate(xs)
    Ytr = np.concatenate(ys)
    del X, Y
    Xte, Yte = load_CIFAR_batch(os.path.join(ROOT, 'test_batch'))
    return Xtr, Ytr, Xte, Yte

In [21]:
class Conv_Neural(nn.Module):
    def __init__(self):
        super(Conv_Neural, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=6, kernel_size=5)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5)
        self.fc1 = nn.Linear(in_features=(16 * 5 * 5), out_features=120)
        self.fc2 = nn.Linear(in_features=120, out_features=84)
        self.fc3 = nn.Linear(in_features=84, out_features=10)
        
    def forward_prop(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
    def plot_loss(self,J_loss):
        #plot loss graph
        plt.plot(J_loss)
        plt.xlabel("No: of Iterations")
        plt.ylabel("Cost")
        plt.show()

In [22]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001)

In [32]:
Dict = {}
Dict = get_CIFAR10_data('cifar-10-batches-py/')

val_data = Dict['X_val']
val_labels = Dict['y_val']

test_data = Dict['X_test']
test_labels = Dict['y_test']

training_data = Dict['X_train']
training_labels = Dict['y_train']

train_x  = torch.from_numpy(training_data).float()
train_y = training_labels.astype(int);
train_y = torch.from_numpy(train_y).float()

test_x  = torch.from_numpy(test_data).float()
test_y = test_labels.astype(int);
test_y = torch.from_numpy(test_y).float()

batch_size = 100
iter = int(len(train_x)/batch_size) 

loss_cal = []
running_loss = 0.0
network = Conv_Neural()
network = network.float()
            
for epoch in range(10):

    print("Loss init:",running_loss)
    print('epoch',epoch)
    k = 0
    running_loss = 0.0

    for i in range(0, iter):

        train_data = train_x[k: k+batch_size]
        train_label = train_y[k: k+batch_size]

        #print(i," : ",train_data.shape)
        inputs = train_data
        labels = train_label

        optimizer.zero_grad()

        outputs = network(inputs)

        loss = criterion(outputs, labels.long())

        running_loss += loss

        loss.backward()

        optimizer.step()

        k += batch_size
                
    loss_cal.append(running_loss)

Loss init: 0.0
epoch 0


NotImplementedError: 

Loss init: 0.0
epoch 0


NotImplementedError: 